# ML.Net - ScaleData

In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

## Using C# Class

In [6]:
using System;
using Microsoft.ML;
using Microsoft.ML.Data;

## Declare data-classes for input data and predictions

In [7]:
public class HousingData
{
    [LoadColumn(0)]
    public float Longitude { get; set; }

    [LoadColumn(1)]
    public float Latitude { get; set; }

    [LoadColumn(2)]
    public float HousingMedianAge { get; set; }

    [LoadColumn(3)]
    public float TotalRooms { get; set; }

    [LoadColumn(4)]
    public float TotalBedrooms { get; set; }

    [LoadColumn(5)]
    public float Population { get; set; }

    [LoadColumn(6)]
    public float Households { get; set; }

    [LoadColumn(7)]
    public float MedianIncome { get; set; }

    [LoadColumn(8), ColumnName("Label")]
    public float MedianHouseValue { get; set; }

    [LoadColumn(9)]
    public string OceanProximity { get; set; }
}

public class HousingPrediction
{
    [ColumnName("Score")]
    public float PredictedHouseValue { get; set; }
}

## Evaluate

In [8]:
var context = new MLContext();

var data = context.Data.LoadFromTextFile<HousingData>("./datasets/housing/housing.csv", hasHeader: true, separatorChar: ',');

var columns = data.Schema
    .Select(col => col.Name)
    .Where(col => col != "Label" && col != "OceanProximity")
    .Select(col => new InputOutputColumnPair(col, col))
    .ToArray();

var scaling = context.Transforms.NormalizeMinMax(columns);

var dataScaled = scaling.Fit(data).Transform(data);

var features = columns.Select(col => col.OutputColumnName).ToArray();

var preview = dataScaled.Preview();

var pipeline = context.Transforms.Text.FeaturizeText("Text", "OceanProximity")
    .Append(context.Transforms.Concatenate("Features", features))
    .Append(context.Transforms.Concatenate("Features", "Text"))
    .Append(context.Regression.Trainers.LbfgsPoissonRegression());

var model = pipeline.Fit(dataScaled);

var predictionFunc = context.Model.CreatePredictionEngine<HousingData, HousingPrediction>(model);

var prediction = predictionFunc.Predict(new HousingData
{
    Longitude = -122.25f,
    Latitude = 37.85f,
    HousingMedianAge = 55.0f,
    TotalRooms = 1627.0f,
    TotalBedrooms = 235.0f,
    Population = 322.0f,
    Households = 120.0f,
    MedianIncome = 8.3014f,
    OceanProximity = "NEAR BAY"
});

Console.WriteLine($"Prediction - {prediction.PredictedHouseValue}");

Console.ReadLine();

Prediction - 259222,4
